In [2]:
from __future__ import annotations
from SPARQLWrapper import SPARQLWrapper, JSON
import os
import sys
import datetime
import dateutil
import json
import pathlib
import time
from collections import defaultdict
from typing import Any, Callable, Dict, Iterable, List, Tuple, Optional, Set, Union

import elasticsearch
import pandas as pd
import requests
import requests_cache
import spacy
import yfinance as yf
from dotenv import load_dotenv
from elasticsearch_dsl import connections, Document, Date, Keyword, Q, Search, Text, Range, Integer
from flashtext import KeywordProcessor

In [1]:
%cd /workspace/entity_knowledge_in_bert/bert_entity
from vocab import Vocab
from data_loader_wiki import EDLDataset

class args:
    data_version_name = "dummy"
    train_loc_file = "train.loc"
    device = "cpu"
    label_size = 1024
    uncased = True
    train_data_dir = "data"
    wiki_lang_version = "enwiki"
    batch_size = 1
    data_workers = 4
    collect_most_popular_labels_steps = 1
    top_rnns = False
    project = False
    entity_embedding_size = 768
    sparse = True
    out_device = 'cpu'
    finetuning = 3

vocab = Vocab(args)
train_dataset = EDLDataset(
    args, split="train", vocab=vocab, device=args.device, label_size=args.label_size)
iterator = train_dataset.get_data_iter(args, batch_size=args.batch_size, vocab=vocab, train=True)

for x in iterator:
#     batch_token_ids, label_ids, label_probs, eval_mask, _, _, orig_batch, loaded_batch = x
#     print(x)
#     print()
    break

/workspace/entity_knowledge_in_bert/bert_entity


In [7]:
batch_token_ids, label_ids, label_probs, eval_mask, _, _, orig_batch, loaded_batch = x

# batch_token_ids, label_ids
# orig_batch
# eval_mask
# loaded_batch
# label_probs.shape
# label_ids.shape
# batch_token_ids.shape
# label_ids
# for p in label_probs[0]:
#     print(p)
# label_probs[0, 13]
# for p in label_probs[0]:
#     print(p.sum())


In [3]:
from model import Net
model = Net(args, vocab.size())

In [4]:
# print(model)
y = model(batch_token_ids, label_ids, label_probs)

tensor([ 1312, 51457,   184,  ..., 32567, 10045, 44862])


In [80]:
logits, _, y_hat, _, _, enc = y
# logits.shape
# logits
# y_hat


# vocab.size()
label_ids.

torch.Size([1024])

In [9]:
from pytorch_pretrained_bert import BertModel

bert = BertModel.from_pretrained("bert-base-uncased")

In [59]:
bert.eval()
encoded_layers, _ = bert(batch_token_ids)
# batch_token_ids.shape
enc = encoded_layers[-1]

In [57]:
import torch.nn as nn
out = nn.Embedding(num_embeddings=vocab.size(), embedding_dim=768, sparse=True)
out.eval()

Embedding(51459, 768, sparse=True)

In [74]:
# _.shape
# encoded_layers[-1].shape
embeded_labels = out(label_ids) # (1024, 768)
embeded_labels = embeded_labels.transpose(0, 1)  # (768, 1024)
logits = enc.matmul(embeded_labels) # (1, 254, 1024)
# logits.argmax(-1)  # (1,254)
logits[0,0,13]

tensor(41.4637, grad_fn=<SelectBackward>)

In [5]:
import torch
import torch.nn.functional as F
x = torch.randn(1, 5)  # (1, 5)
y = torch.randint(5, (1,), dtype=torch.int64)  # (1, )
loss = F.cross_entropy(x, y)
# loss
# x.shape, y.shape
# x, y

x = torch.randn((1, 5))
y = torch.rand((1, 5))
# F.binary_cross_entropy(torch.sigmoid(x), y)
# x, y
# F.binary_cross_entropy(x, y)
# torch.sigmoid(x)
y
y.to("cpu")

tensor([[0.3413, 0.5543, 0.3464, 0.6351, 0.4112]])

In [3]:
def count_corpus_mentions_cooccurs(mentions: Dict[str, List[str]]) -> Any:
    mention2idx = get_mention2idx_dict(mentions)

    def count_cooccur(bag: Iterable[int], cooccur: Dict[int, Dict[int, int]] = defaultdict(lambda: defaultdict(int))) -> Dict[int, Dict[int, int]]:
        for a in bag:
            for b in bag:
                cooccur[a][b] += 1
        return cooccur

    nlp = spacy.load('en_core_web_sm')
    processor = KeywordProcessor()
    processor.add_keywords_from_dict(mentions)

    sent_cooccur, atk_cooccur = defaultdict(
        lambda: defaultdict(int)), defaultdict(lambda: defaultdict(int))
    atk_bags = []
    for i, hit in enumerate(es.scan_scraper_page("*cnbc*")):
        if i % 100 == 0:
            print(i)
        if i > 1000:
            break
        sents = []
        try:
            sents.append(hit.article_title)
        except:
            pass
        try:
            sents += [sent.text for sent in nlp(hit.article_text).sents]
        except:
            pass

        atk_bag = set()
        for sent in sents:
            kws = processor.extract_keywords(sent, span_info=False)
            sent_bag = [mention2idx[kw] for kw in kws]
            sent_cooccur = count_cooccur(sent_bag, sent_cooccur)
            atk_bag |= set(sent_bag)
        atk_cooccur = count_cooccur(atk_bag, atk_cooccur)
        atk_bags.append(atk_bag)

    return (
        # 在一個句子中，兩個entity同時出現的次數: {a: {a: 4, b: 2, c:1}, b: {...}}
        dict(sent_cooccur),
        # 在一篇文章中，兩個entity同時出現的次數: {a: {a: 4, b: 2, c:1}, b: {...}}
        dict(atk_cooccur),
        # 在一篇文章中出現的entities集合成一個bag: [{a, b, c}, {a, c, f}, ...]
        set(map(frozenset, atk_bags)),)

In [5]:
filepath = "/workspace/entity_knowledge_in_bert/app/outputs/wiki_mentions.json"

with open(filepath, encoding='utf-8') as f:
    mentions = json.load(f)

mentions

FileNotFoundError: [Errno 2] No such file or directory: '/workspace/entity_knowledge_in_bert/app/outputs/wiki_mentions.json'